In [365]:
import json
import os
import pandas as pd

Read english job ads

In [366]:
df = pd.read_json('/Users/cwestrup/thesis/data/sanoma/oikotie/jobs-en-2015-12-04T11:45:53.720971.json')

Select name and job_description

In [368]:
df = df.loc[:,['name','job_description']]
df.head()

,name,job_description
0,"Sandbox Software Engineer (ATP), Backend autom...",Coders needed<br />\nWe are looking for passio...
1,"Channel Manager, Cyber Security Services Nordics",F-Secure offers an exciting and challenging po...
2,Experienced Software Engineer in Test (ATP),Coders needed<br />\nWe’re looking for softwar...
3,"Experienced Software Engineer (ATP), Network S...",We’re looking for software engineers who can d...
4,Big Data & Machine Learning Software Engineer ...,We are looking for passionate Software Enginee...


In [353]:
df.job_description[0]

'Coders needed<br />\nWe are looking for passionate Software Engineer to develop advanced virtualized sandbox execution and analysis environment. In this position, you will work as part of a team creating a world-class runtime behavioral analysis platform. The right candidate will have a deep understanding of virtualization and virtual machine instrumentation. <br />\n<br />\nWhat’s in it for me?<br />\n<br />\nAs part of the ATP team, you’ll be pioneering something new for F-Secure, in a field that is experiencing rapid growth. Our goal is to become a market leader in the European cyber-security space. You’ll be working alongside experts in a dynamic and international environment. As a new project inside F-Secure, you’ll get to define the way you work and shape the future of this exciting new endeavor. We offer flexibility in the workplace and a vibrant company culture. This position offers a diverse range of paths towards success and career growth.<br />\n<br />\nHow to stand out as 

Write out all jobs with names and full description text

In [370]:
df.index.name = 'ad_id'
df.to_csv('alljobs.csv')

### Creating text chunks (paragraphs)

Select job descriptions

In [371]:
df_chunks = df.loc[:,['job_description']]

Replace '\n' by html br

In [372]:
to_spaces = '\n'
df_chunks = df_chunks.replace({'\n': '<br>'}, regex=True)

Replace html 'br' variants with 'BREAK'

In [373]:
to_paragraph_breaks = '(<br />)+|(<br/>)+|(<br>)+|(</br>)+'
df_chunks = df_chunks.replace(to_replace=to_paragraph_breaks, value='<BREAK>', regex=True)

Add break after html p tags

In [374]:
to_paragraph_breaks = '</p>|</ p>'
df_chunks = df_chunks.replace(to_replace=to_paragraph_breaks, value='</p><BREAK>', regex=True)

Remove all multiple occurences of BREAK tag

In [375]:
df_chunks = df_chunks.replace(to_replace='(<BREAK>)+', value='<BREAK>', regex=True)

Split rows by BREAK tags and stack up to a new DataFrame

In [376]:
df_chunks = df_chunks['job_description'].str.split('<BREAK>').apply(pd.Series)
df_chunks = df_chunks.stack().to_frame()
df_chunks.index.names = ['ad_id', 'chunk_id']
df_chunks.columns = ['content']

In [377]:
df_chunks.head(10)

content
ad_id chunk_id                                                   
0     0                                             Coders needed
      1         We are looking for passionate Software Enginee...
      2                                      What’s in it for me?
      3         As part of the ATP team, you’ll be pioneering ...
      4                        How to stand out as an applicant? 
      5             The following experience is an absolute must:
      6         • Extensive prior experience with hypervisors ...
      7         • We value prior experience integrating the ab...
      8         • Understanding of quality practices, continuo...
      9         • We value experience working with public or p...

In [378]:
df_chunks.to_csv('chunks.csv')

Import into MongoDB with:

     mongoimport -d thesis -c ads --type csv --file ./alljobs.csv --headerline
     mongoimport -d thesis -c chunks --type csv --file ./chunks.csv --headerline